In [1]:
import sys
sys.path.append('/workspace/Documents')
# imports
import os, sys

# third party imports
import numpy as np 

import pandas as pd
import random
import nibabel as nb
import cv2
import ast

import Diffusion_motion_field.Build_lists.Build_list as Build_list
import Diffusion_motion_field.functions_collection as ff
import Diffusion_motion_field.Data_processing as Data_processing

main_path = '/mnt/camca_NAS/4DCT'

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


check the dvf difference between low EF and high EF

In [22]:
timeframe_info = pd.read_excel('/mnt/camca_NAS/4DCT/Patient_lists/patient_list_final_selection_timeframes.xlsx')
print(timeframe_info.shape)
ef_low_group = timeframe_info[(timeframe_info['EF_sampled']<=0.60) & (timeframe_info['EF_sampled']>=0.50)]
ef_high_group = timeframe_info[(timeframe_info['EF_sampled']>=0.60) & (timeframe_info['EF_sampled']<=0.70)]
print('Low EF group:', len(ef_low_group), 'High EF group:', len(ef_high_group))

(317, 11)
Low EF group: 43 High EF group: 99


In [24]:
# randomly pick one from each group
result = []
for i in range(0,100):
    ef_low_case = ef_low_group.sample(1)
    ef_high_case = ef_high_group.sample(1)

    patient_id_low = ef_low_case['patient_id'].values[0]
    patient_class_low = ef_low_case['patient_class'].values[0]

    patient_id_high = ef_high_case['patient_id'].values[0]
    patient_class_high = ef_high_case['patient_class'].values[0]

    print('low EF:', patient_id_low, 'class:', patient_class_low, ' EF:', ef_low_case['EF_sampled'].values[0])
    print('high EF:', patient_id_high, 'class:', patient_class_high, ' EF:', ef_high_case['EF_sampled'].values[0])

    mvf_folder_low = os.path.join(main_path,'models/VAE_embed3/pred_mvf',patient_class_low, patient_id_low,'epoch54')
    row = timeframe_info[timeframe_info['patient_id'] == patient_id_low]
    sampled_time_frame_list = ast.literal_eval(row['sampled_time_frame_list'].iloc[0])
    normalized_time_frame_list = ast.literal_eval(row['normalized_time_frame_list_copy'].iloc[0])

    picked_tf_normalized = [0.1,0.3,0.5,0.7,0.9]
    picked_tf = [sampled_time_frame_list[normalized_time_frame_list.index(picked_tf_normalized[iii])] for iii in range(0,len(picked_tf_normalized))]   

    mvf_low = np.zeros((5,40,40,24,3))
    for iii in range(0,len(picked_tf)):
        mvf_low[iii] = nb.load(os.path.join(mvf_folder_low,'pred_latent_tf'+str(picked_tf[iii])+'.nii.gz')).get_fdata()

    # mvf_low = Data_processing.cutoff_intensity(mvf_low, cutoff_low = -30, cutoff_high = 30)
    # mvf_low = Data_processing.normalize_image(mvf_low, normalize_factor = 'equation', image_max = 30, image_min = -30, invert = False)

    mvf_folder_high = os.path.join(main_path,'models/VAE_embed3/pred_mvf',patient_class_high, patient_id_high,'epoch54')
    row = timeframe_info[timeframe_info['patient_id'] == patient_id_high]
    sampled_time_frame_list = ast.literal_eval(row['sampled_time_frame_list'].iloc[0])
    normalized_time_frame_list = ast.literal_eval(row['normalized_time_frame_list_copy'].iloc[0])
    picked_tf_normalized = [0.1,0.3,0.5,0.7,0.9]
    picked_tf = [sampled_time_frame_list[normalized_time_frame_list.index(picked_tf_normalized[iii])] for iii in range(0,len(picked_tf_normalized))]
    mvf_high = np.zeros((5,40,40,24,3))
    for iii in range(0,len(picked_tf)):
        mvf_high[iii] = nb.load(os.path.join(mvf_folder_high,'pred_latent_tf'+str(picked_tf[iii])+'.nii.gz')).get_fdata()
    # mvf_high = Data_processing.cutoff_intensity(mvf_high, cutoff_low = -30, cutoff_high = 30)
    # mvf_high = Data_processing.normalize_image(mvf_high, normalize_factor = 'equation', image_max = 30, image_min = -30, invert = False)
        

    mvf_low_focus = mvf_low[2,...]
    mvf_high_focus = mvf_high[2,...]
    # print(np.max(mvf_low_focus), np.min(mvf_low_focus))
    # print(np.max(mvf_high_focus), np.min(mvf_high_focus))


    # calculate the mse 
    mse = np.mean((mvf_high_focus - mvf_low_focus)**2)
    ef_difference = ef_high_case['EF_sampled'].values[0] - ef_low_case['EF_sampled'].values[0]

    result.append([patient_id_low, patient_id_high,ef_difference, mse])
    df = pd.DataFrame(result, columns=['patient_id_low', 'patient_id_high', 'ef_difference', 'mse'])
    df.to_excel('/mnt/camca_NAS/4DCT/Patient_lists/mvf_difference_due_to_ef_difference2.xlsx', index=False)




low EF: CVC1912111149 class: Normal  EF: 0.5723988479214014
high EF: CVC1804131105 class: Normal  EF: 0.6239205754183164
low EF: CVC1901241202 class: Abnormal  EF: 0.5829048290647539
high EF: CVC1804111039 class: Normal  EF: 0.6587390379372162
low EF: CVC1908231346 class: Abnormal  EF: 0.5605716712877067
high EF: CVC1912160957 class: Normal  EF: 0.6369139611865431
low EF: CVC1812051109 class: Normal  EF: 0.5993816066905345
high EF: CVC1803301406 class: Normal  EF: 0.6492419831360118
low EF: CVC1905071402 class: Normal  EF: 0.5729074680397245
high EF: CVC1902151307 class: Normal  EF: 0.6172655141007443
low EF: CVC1905281145 class: Abnormal  EF: 0.5899938513316721
high EF: CVC1804131412 class: Normal  EF: 0.689354309997939
low EF: CVC1812171232 class: Normal  EF: 0.5128577469503779
high EF: CVC1803301406 class: Normal  EF: 0.6492419831360118
low EF: CVC1909121046_AN131 class: Normal  EF: 0.5614378292491806
high EF: CVC1810081017 class: Normal  EF: 0.6505855632223605
low EF: CVC2006300933